## E-09: Movielens 영화 추천시스템 만들기 
* 평가지표  
   1. CSR matrix가 정상적으로 만들어졌다.  
      사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
   
   2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
      사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
   
   3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
      MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.


* 활용할 데이터셋 :  추천시스템의 MNIST라고 부를만한 Movielens 데이터

* Movielens  
 - 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다  
    **MovieLens 1M Dataset 사용을 권장**합니다.  
 - **별점** 데이터는 대표적인 explicit 데이터입니다.  
    하지만 **implicit 데이터로 간주**하고 테스트해볼 수 있습니다.
 - 별점을 **시청횟수**로 해석해서 생각하겠습니다.
 - 유저가 **3점 미만**으로 준 데이터는 **선호하지 않는다**고 가정하고 제외하 겠습니다.

In [1]:
# Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 
#심볼릭 링크로 개인 storage에 연결
! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

### 1. 데이터 불러오기 및 simple 전처리

#### 1. 데이터 가져오기

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### 2. 전처리  

##### 2-1) 3점 미만 데이터 미선호 데이터로 간주 = 3점 이상만 데이터에 남기기

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


##### 2-2) 유저가 몇번 봤는지 : ratings  -> count 로  
데이터셋 이름이랑 겹침

In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

##### 2-3) 영화 제목 확인해보기

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


##### 2-4) 영화제목 소문자로 바꾸기

In [6]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


##### 2-5) 영화 제목 들어간 ratings 데이터 만들기

In [7]:
data = ratings.join(movies.set_index('movie_id'), on='movie_id')

data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy


In [8]:
del data['timestamp']
#del data['genre']
data.head()

,user_id,movie_id,count,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,my fair lady (1964),Musical|Romance
3,1,3408,4,erin brockovich (2000),Drama
4,1,2355,5,"bug's life, a (1998)",Animation|Children's|Comedy


### 2. 데이터 분석하기

In [9]:
#유저수
data['user_id'].nunique()

6039

In [10]:
# 영화 개수
data['movie_id'].nunique()

3628

In [11]:
#인기 많은 영화
data_ratings = data.groupby('title')['user_id'].count()
#ascending=False : 내림 차순으로 순위 매김
#rank() : 그냥 순위 정하는 메소드
data_ratings.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [12]:
# 유저별 몇개의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('movie_id')['user_id'].count()
user_count.describe()

count    3628.000000
mean      230.561742
std       355.596393
min         1.000000
25%        23.000000
50%        87.000000
75%       285.000000
max      3211.000000
Name: user_id, dtype: float64

### 3. 영화를 5가지 골라서 data에 넣기 
처음 가입하는 사용자의 취향과 유사한 영화 정보를 5개 이상 입력받는 과정

In [45]:
# 영화 genre 키워드로 검색
movies[movies['genre'].str.lower().str.contains('comedy|romance', regex=False)]

,movie_id,title,genre
2,3,grumpier old men (1995),Comedy|Romance
6,7,sabrina (1995),Comedy|Romance
38,39,clueless (1995),Comedy|Romance
63,64,two if by sea (1996),Comedy|Romance
67,68,french twist (gazon maudit) (1995),Comedy|Romance
...,...,...,...
3679,3748,"match, the (1999)",Comedy|Romance
3759,3829,mad about mambo (2000),Comedy|Romance
3804,3874,"couch in new york, a (1996)",Comedy|Romance
3815,3885,love & sex (2000),Comedy|Romance


In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('forrest gump', regex=False)]

,movie_id,title,genre
352,356,forrest gump (1994),Comedy|Romance|War


In [14]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('sleepless', regex=False)]

,movie_id,title,genre
535,539,sleepless in seattle (1993),Comedy|Romance


In [15]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('true lies', regex=False)]

,movie_id,title,genre
376,380,true lies (1994),Action|Adventure|Comedy|Romance


In [16]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('lady vanishes', regex=False)]

,movie_id,title,genre
2139,2208,"lady vanishes, the (1938)",Comedy|Mystery|Romance|Thriller


In [17]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('groundhog day', regex=False)]

,movie_id,title,genre
1245,1265,groundhog day (1993),Comedy|Romance


In [19]:
# 좋아하는 영화의 id 리스트
my_favoite_movie_id = [356, 539, 380, 2208, 1265]

# id 리스트를 title 리스트로 변환해준다
my_favorite_movie = []
for mid in my_favoite_movie_id:
    my_favorite_movie.append(list(movies[movies['movie_id']==mid]['title'])[0])
    
# id 리스트를 genre 리스트로 변환해준다
#my_favorite_genre = []
#for mid in my_favoite_movie_id:
#    my_favorite_genre.append(list(movies[movies['movie_id']==mid]['genre'])[0])

# '923'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_watchlist = pd.DataFrame({'user_id': ['gayeong']*5, 'movie_id': my_favoite_movie_id, 'title': my_favorite_movie, 'count':[5]*5})

if not data.isin({'user_id':['gayeong']})['user_id'].any():  # user_id에 'eunyoeng'이라는 데이터가 없다면
    data = data.append(my_watchlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data = data[['user_id', 'movie_id', 'title', 'count']]
data.tail(10)

,user_id,movie_id,title,count
1000203,6040,1090,platoon (1986),3
1000205,6040,1094,"crying game, the (1992)",5
1000206,6040,562,welcome to the dollhouse (1995),5
1000207,6040,1096,sophie's choice (1982),4
1000208,6040,1097,e.t. the extra-terrestrial (1982),4
0,gayeong,356,forrest gump (1994),5
1,gayeong,539,sleepless in seattle (1993),5
2,gayeong,380,true lies (1994),5
3,gayeong,2208,"lady vanishes, the (1938)",5
4,gayeong,1265,groundhog day (1993),5


## 왜 counts로 하면 안될까?
왜 장르는 안될까?

# 본인이 좋아하는 영화 5개 데이터 넣기
my_favorite_title = ['lalaland' , 'soul' ,'note book' ,'before sunrise' ,'closer']
my_favorite_movie_id = {3628, 3629, 3630, 3631, 3632}
my_favorite_genre = {'Romance', 'Drama', 'Romance', 'Romance', 'Romance'}
# 'gayeong'이라는 user_id가 위 영화를 4번씩 봤다고 가정
my_playlist = pd.DataFrame({'user_id': ['923']*5, 'movie_id' : my_favorite_movie_id,  'counts':[4]*5,'title': my_favorite_title , 'genre' : my_favorite_genre})

# user_id에 'gayeong'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다.
if not data.isin({'user_id':['923']})['user_id'].any():  
    data = data.append(my_playlist)                            

data = data[['user_id', 'movie_id', 'title', 'counts', 'genre']]
data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

#	user_id	movie_id	counts	title	genre

### 4. CSR 매트릭스 생성   
CSR(Compressed Sparse Row) Matrix  
sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는  
데이터 값과 좌표 정보만으로 구성 -> 메모리 사용량 최소화   

##### 1. 매트릭스 생성 위한 인덱싱

In [20]:
from scipy.sparse import csr_matrix
#user, title 찾기 (중복없이)
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

#user, movie 각각 인덱싱 하기
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [21]:
#user_id 모두 인덱싱 하기
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [22]:
#title 모두 인덱싱 하기
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

title column indexing OK!!


In [23]:
data

,user_id,movie_id,title,count
0,0,1193,0,5
1,0,661,1,3
2,0,914,2,3
3,0,3408,3,4
4,0,2355,4,5
...,...,...,...,...
0,6039,356,160,5
1,6039,539,666,5
2,6039,380,138,5
3,6039,2208,1324,5


##### 2. csr 매트릭스 생성

In [24]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5. als_model = AlternatingLeastSquares 모델 구성 및 학습  

시청횟수 = count = implicit 데이터로 간주  

* Matrix Factorization 모델을 implicit 패키지를 사용하여 학습

* implicit 패키지  
  암묵적(implicit) dataset을 사용하는 다양한 모델을  
  굉장히 빠르게 학습할 수 있는 패키지  
  
* als(AlternatingLeastSquares) 모델  
  Matrix Factorization에서 쪼개진 두 Feature Matrix를  
  한꺼번에 훈련하는 것은 잘 수렴하지 않음  
  -> 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는  
     AlternatingLeastSquares 방식 사용  

##### 1. 모델 구성

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* AlternatingLeastSquares 클래스의 __init__ 파라미터  
  1) factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지  
  2) regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지  
  3) use_gpu : GPU를 사용할 것인지  
  4) iterations : epochs와 같은 의미입니다.  
    데이터를 몇 번 반복해서 학습할 것인지  
  - 1,4를 늘릴수록 학습을 잘 하게 되지만 과적합의 우려가 있다.

In [26]:
# Implicit AlternatingLeastSquares 모델의 선언
# factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줌
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

##### 2. 모델 학습

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6. 내가 선호하는 5가지 영화 중 하나, 그외의 영화 하나를 골라  
### 예측된 나의 선호도를 파악하기

#### 1.  gayeong 사용자 벡터 before sunrise의 벡터를 어떻게 만들고 있는지 확인

In [42]:
gayeong, forrest = user_to_idx['gayeong'], movie_to_idx['forrest gump (1994)']
gayeong_vector, forrest_vector = als_model.user_factors[gayeong], als_model.item_factors[forrest]

In [39]:
print(gayeong_vector.shape)
gayeong_vector

(100,)


array([ 6.1388355e-01, -1.9906387e-01,  2.9857986e-02,  1.5229097e-01,
        6.6105038e-01, -1.9354695e-01, -3.3574861e-01,  3.0278969e-01,
       -4.9996685e-02,  3.5606261e-02, -6.9815403e-01,  1.2824778e-01,
       -4.1147920e-01,  1.4879268e-01,  7.3970234e-01,  2.1792513e-01,
       -8.2292311e-02, -3.4573019e-01, -3.1053069e-01,  6.4462370e-01,
       -1.9620214e-01, -9.1814399e-01, -4.5855302e-01, -9.2134792e-01,
       -3.6663625e-02, -4.8978623e-02,  1.8357427e-01,  1.3090259e-02,
        1.3395137e-01, -6.6258317e-01,  5.4006692e-02,  1.0839574e-01,
        4.8665145e-01,  3.4941998e-01,  4.1014305e-01, -6.2200099e-01,
       -2.5079858e-01, -1.4500964e-01,  1.2835301e+00, -8.5613114e-01,
        3.0585292e-01,  6.1266845e-01, -6.9250286e-01,  1.1638041e+00,
        3.0540836e-01,  9.7258084e-02,  2.0344917e-02, -1.6770691e-01,
       -2.9928157e-01, -1.2364511e-01, -1.5636969e-01,  5.9709597e-01,
       -6.5246008e-02, -8.2345802e-01,  6.9843614e-01,  5.4169830e-02,
      

In [40]:
print(forrest_vector.shape)
forrest_vector

(100,)


array([ 0.0019017 ,  0.02557408, -0.00457458, -0.00311147,  0.03098644,
       -0.00577148,  0.00325144,  0.03311893, -0.00878399,  0.02342585,
       -0.02577823,  0.01321712,  0.00285468,  0.0343077 ,  0.03524889,
        0.01750071,  0.00109344,  0.00217265, -0.00944461,  0.02876705,
       -0.01207686, -0.02525539, -0.00112479, -0.04280132, -0.00104615,
       -0.01352699, -0.00880572,  0.00188296,  0.01891545, -0.02114863,
       -0.00317265,  0.01804841,  0.02351053,  0.00536738,  0.02292667,
        0.01787689,  0.00084576, -0.01339651,  0.05871806, -0.03627729,
        0.03978118,  0.03068557, -0.02339378,  0.00492047,  0.02331992,
        0.00103232,  0.01129805,  0.00012453,  0.01573024, -0.00638882,
       -0.00106527,  0.02853175, -0.00872011, -0.0191827 ,  0.03061148,
        0.00372741, -0.0137231 ,  0.0370957 ,  0.03175524, -0.01851245,
        0.014066  , -0.00109261,  0.01002583,  0.03360716,  0.01241174,
        0.01344597, -0.03920833, -0.02350646, -0.02668619,  0.00

In [41]:
# gayeong과 forrest gump (1994)를 내적하는 코드
np.dot(gayeong_vector,forrest_vector)

0.7116494

In [35]:
gayeong, seattle = user_to_idx['gayeong'], movie_to_idx['sleepless in seattle (1993)']
gayeong_vector, seattle_vector = als_model.user_factors[gayeong], als_model.item_factors[seattle]

np.dot(gayeong_vector,seattle_vector)

0.3922202

In [36]:
gayeong, true = user_to_idx['gayeong'], movie_to_idx['true lies (1994)']
gayeong_vector, true_vector = als_model.user_factors[gayeong], als_model.item_factors[true]

np.dot(gayeong_vector,true_vector)

0.39196187

In [37]:
gayeong, vanishes = user_to_idx['gayeong'], movie_to_idx['lady vanishes, the (1938)']
gayeong_vector, vanishes_vector = als_model.user_factors[gayeong], als_model.item_factors[vanishes]

np.dot(gayeong_vector,vanishes_vector)

0.14825507

In [38]:
gayeong, groundhog_day = user_to_idx['gayeong'], movie_to_idx['groundhog day (1993)']
gayeong_vector, groundhog_day_vector = als_model.user_factors[gayeong], als_model.item_factors[groundhog_day]

np.dot(gayeong_vector,groundhog_day_vector)

0.58212316

##### 2. 아직 보지 않은 american beauty (1999)

In [72]:
# 아직 듣지 못한 데이터 =  princess bride, the (1987)
#princess bride, the (1987)에 대한 선호도 확인해보자

american = movie_to_idx['american beauty (1999)']
american_vector = als_model.item_factors[american]
np.dot(gayeong_vector, american_vector)

0.089406736

### 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [73]:
# 좋아하는 영화와 비슷한 영화의 인덱스 얻기
favorite_movie = 'sleepless in seattle (1993)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(666, 0.99999976),
 (336, 0.8816539),
 (626, 0.86150604),
 (340, 0.7900148),
 (579, 0.7816618),
 (1151, 0.7681574),
 (1088, 0.7204677),
 (516, 0.6896739),
 (508, 0.6849929),
 (1212, 0.6467601),
 (821, 0.6424054),
 (466, 0.6383602),
 (1234, 0.6086361),
 (523, 0.5959443),
 (613, 0.59227556)]

In [74]:
# 추천받은 영화의 인덱스를 영화 제목으로 얻기
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['sleepless in seattle (1993)',
 'pretty woman (1990)',
 "you've got mail (1998)",
 "my best friend's wedding (1997)",
 'while you were sleeping (1995)',
 'notting hill (1999)',
 'truth about cats & dogs, the (1996)',
 'sabrina (1995)',
 'ghost (1990)',
 'tin cup (1996)',
 'mrs. doubtfire (1993)',
 'wedding singer, the (1998)',
 'dave (1993)',
 'michael (1996)',
 'four weddings and a funeral (1994)']

#### 2. 비슷한 영화 추천받는 함수 생성
인덱스로 얻고, 인덱스를 영화 제목으로 출력해주는 함수

In [59]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [70]:
get_similar_movie('sleepless in seattle (1993)')

#동일 결과 도출

['sleepless in seattle (1993)',
 'pretty woman (1990)',
 "you've got mail (1998)",
 "my best friend's wedding (1997)",
 'while you were sleeping (1995)',
 'notting hill (1999)',
 'truth about cats & dogs, the (1996)',
 'sabrina (1995)',
 'ghost (1990)',
 'tin cup (1996)',
 'mrs. doubtfire (1993)',
 'wedding singer, the (1998)',
 'dave (1993)',
 'michael (1996)',
 'four weddings and a funeral (1994)']

### 8. 내가 가장 좋아할만한 영화 추천받기

In [75]:
user = user_to_idx['gayeong']
# recommend에서는 user*item CSR Matrix를 받습니다.
#filter_already_liked_items: 유저가 이미 평가한 아이템(아티스트)는 제외
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(336, 0.3986169),
 (613, 0.36711675),
 (204, 0.31483555),
 (626, 0.3039292),
 (508, 0.29728234),
 (154, 0.28043276),
 (40, 0.2753257),
 (236, 0.26655513),
 (340, 0.24650499),
 (20, 0.24222419),
 (1151, 0.24191655),
 (1200, 0.24083772),
 (1234, 0.24001119),
 (126, 0.23537809),
 (27, 0.2260594),
 (579, 0.21699317),
 (821, 0.21666765),
 (466, 0.2137393),
 (156, 0.20763507),
 (175, 0.20298736)]

In [76]:
[idx_to_movie[i[0]] for i in movie_recommended]

['pretty woman (1990)',
 'four weddings and a funeral (1994)',
 'clueless (1995)',
 "you've got mail (1998)",
 'ghost (1990)',
 'as good as it gets (1997)',
 'toy story (1995)',
 'speed (1994)',
 "my best friend's wedding (1997)",
 'pleasantville (1998)',
 'notting hill (1999)',
 'american president, the (1995)',
 'dave (1993)',
 'shakespeare in love (1998)',
 'titanic (1997)',
 'while you were sleeping (1995)',
 'mrs. doubtfire (1993)',
 'wedding singer, the (1998)',
 'mask of zorro, the (1998)',
 'men in black (1997)']

##### 2. 기여도 확인

In [78]:
# AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 
# 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도 확인 가능

recommended = movie_to_idx['pretty woman (1990)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('sleepless in seattle (1993)', 0.18087009137502558),
 ('forrest gump (1994)', 0.0923563917956461),
 ('true lies (1994)', 0.07128192338222658),
 ('groundhog day (1993)', 0.04846372761608229),
 ('lady vanishes, the (1938)', -4.8490559681797115e-05)]

* pretty woman (1990)는 comedy|romance 임  
 해당 장르에 대한 영화가 'sleepless in seattle (1993) 가장 유사도가 높다  
 
* lady vanishes는 스릴러, 로맨스,액션,코미디 장르를 가지고 있다  
  pretty woman (1990)는 comedy|romance여셔, 기여도가 낮게 나온것 같음  

## 회고

#### 💚루브릭
1. CSR matrix가 정상적으로 만들어졌다.
   사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
   추천받은 영화들이 wqr, romance,comedy 장르의 영화들임
   사용자와 아이템 벡터 내적수치가 의미있게 형성되었다고 판단되어짐  

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
   MF모델이 예측한 유저 선호도 및 아이템간 유사도에 대해 생각해보았을때  
   대부분의 영화가 로코 장르물이었기 때문에 코미디 로맨스를 좋아하는 나로서는 유의미한 추천이였다.   
   기여도를 확인해볼때도 스릴러 장르 중 하나인 lady vanishes이 기여도가 가장 낮고, 로맨스 코미디 장르인 lady vanishes가 가장 많은 기여도를 가진다. 
   
   
#### 💚더 개선하고 싶은 점  
1. 유사도와 기여도를 더 높여보고 싶다  
   데이터 수가 충분한 학습을 할정도로 많이 수집되지 않았던 것 같고  
   선호도를 표현이 '시청횟수'밖에 없기 때문에  
   조금 더 다양한 암묵적/명시적 평가 척도들을 살펴보고 결정하고 싶다  
   
   
2. 시각화  


3. 최근 영화 데이터셋  
   좋아하는 영화 선정할때 데이터 셋에서 아는 영화가 많지가 않아서 힘들었다  
   최근 영화 데이터셋도 함께 추가하여 살펴보고, 추천 시스템이 넷플릭스의  
   기능과는 어떤게 다른지 등에 대한 분석 및 평가를 진행해보고 싶다.
   
#### 💚노드 내용 외 배운점  
* 시각화의 중요성  
  모델 학습할때 어느정도 잘 학습 되고 있는지 알아보기 어려웠고  
  유사도나 기여도를 시각화하여 본다면 조금 더 직관적인 의사결정이  
  가능했을 것이라고 기대된다  
  때문에 시각화 하는 것에 대해서 많은 노력을 기울이고자 다짐했다

   